#### NOTES
- Concat all docs for each politician before modelling?
- UMAP to lower dimensions before calculating centroid?

#### TO DO
- Modify tensor utils so we can explore the embeddings including party vecs

In [15]:
!pip uninstall politician2vec -y

Found existing installation: politician2vec 0.0.1
Uninstalling politician2vec-0.0.1:
  Successfully uninstalled politician2vec-0.0.1


In [16]:
!pip install --upgrade git+ssh://git@github.com/mathiasbruun/politician2vec.git

  Cloning ssh://****@github.com/mathiasbruun/politician2vec.git to /private/var/folders/8q/02vc3fzn3r1fv7wzycyspjz80000gn/T/pip-req-build-85eba12g
  Running command git clone -q 'ssh://****@github.com/mathiasbruun/politician2vec.git' /private/var/folders/8q/02vc3fzn3r1fv7wzycyspjz80000gn/T/pip-req-build-85eba12g
  Created wheel for politician2vec: filename=politician2vec-0.0.1-py3-none-any.whl size=26678 sha256=12f84fc96cc404218f84044cb0936d46bd655f1cfa796d1e2d3949912a626f8e
  Stored in directory: /private/var/folders/8q/02vc3fzn3r1fv7wzycyspjz80000gn/T/pip-ephem-wheel-cache-70hck797/wheels/ac/44/62/b0b3ddf2882cd1b1d1cc4e060c5c525b951ae01496d65cd472
Successfully built politician2vec


In [53]:
from politician2vec.Politician2Vec import Politician2Vec
from politician2vec.utils import *
import pickle
import pandas as pd
import numpy as np
import multiprocessing
available_workers = multiprocessing.cpu_count()

In [54]:
test_data_path = 'data/clean/combined/subset_party_imputed.p'

In [55]:
with open(test_data_path, 'rb') as p:
    test_data = pickle.load(p)

In [56]:
test_data.shape

(349594, 7)

In [57]:
tw_data = test_data.loc[
    test_data['source'] == 'twitter'
]

In [58]:
tw_data['party'].value_counts(dropna = False)

S      54566
V      37099
EL     27629
RV     22041
SF     21593
ALT    18070
LA     14566
KF     13241
DF     12363
FG      4196
NB      1834
M        157
K          2
Name: party, dtype: int64

In [62]:
grouped_docs = tw_data.groupby(['full_name', 'party'])['doc'].apply('. '.join).reset_index()

In [66]:
#docs = [doc for doc in tw_data.doc]
docs = [doc for doc in grouped_docs.doc]
parties = np.array([party for party in grouped_docs.party])

In [67]:
pol2vec_model = Politician2Vec(
    documents = docs,
    custom_clusters = parties,
    tokenizer = preproc_docs,
    embedding_model = 'doc2vec',
    min_count = 50,
    speed = 'fast-learn',
    workers = available_workers
    #hdbscan_min_cluster_size = 15,
    #workers=4,
    #doc2vec_vector_size = 300,
    #doc2vec_window = 8,
    #doc2vec_samples_threshold = 1e-5
)

2022-11-12 09:32:45,132 - politician2vec - INFO - Pre-processing documents for training
2022-11-12 09:33:17,969 - politician2vec - INFO - Creating joint document/word embedding
2022-11-12 09:38:13,886 - politician2vec - INFO - ONLY creating UMAP projection, NOT HDBSCAN!!!
2022-11-12 09:38:22,483 - politician2vec - INFO - Finding topics


In [68]:
def inspect_topic(top2vec, topic_idx, n_docs=None, query_substr=None):
    '''
    Print top words and top docs for a given
    topic.
    -------
    manual_num (int):  manually assigned topic number (i.e. 1-indexed).
    
    n_docs (int, optional): n top documents to print for a given topic.
        Default is to print all docs within a given topic.
    
    query_substr (str, optional): if specified, only documents containing
        this substring will be printed. Cannot be specified with n_docs,
        as this would return only results within a subset of topic docs.
    '''

    num_topics = top2vec.get_num_topics()
    topic_words, word_scores, topic_nums = top2vec.get_topics(num_topics)

    # Get topic sizes so we know max n docs
    topic_sizes, topic_nums = top2vec.get_topic_sizes()
    docs_to_return = topic_sizes[topic_idx]

    # Override n docs to return, if specified
    if n_docs:
        docs_to_return = n_docs

    # Get docs for input topic id
    documents, document_scores, document_ids = top2vec.search_documents_by_topic(
        topic_num=topic_idx,
        num_docs=docs_to_return
        )

    # Limit output to docs containign certain substring, if specified
    if query_substr and n_docs:
        raise Exception('Please do NOT specify n_docs with substring query!\nOtherwise the search is only carried out for a subset of topic docs.')
    
    # Throw exception if substring query attempted on subset of docs!
    elif query_substr:
        documents = [doc for doc in documents if query_substr in doc.lower()]

    # Print output
    print('--- TOP 50 WORDS ---\n', topic_words[topic_idx], '\n')

    print(f'--- TOP {docs_to_return} DOCS. SUBSTRING QUERY: {query_substr} (n = {len(documents)}) ---\n', documents)

In [69]:
word_vectors = pol2vec_model.model.wv

In [81]:
word_vectors.most_similar(positive = ['minister'])

[('to', 0.5383712649345398),
 ('ministre', 0.5155658721923828),
 ('danish', 0.5089468359947205),
 ('a', 0.49930647015571594),
 ('denmark', 0.4918190836906433),
 ('with', 0.48776963353157043),
 ('on', 0.48716092109680176),
 ('sagen', 0.48335105180740356),
 ('’', 0.4816931188106537),
 ('and', 0.477433443069458)]

In [84]:
inspect_topic(pol2vec_model, 12, n_docs=1, query_substr=None)

--- TOP 50 WORDS ---
 ['moderaterne' 'moderaternes' 'folkepensionen' 'løsningerne'
 'sundhedsudspil' 'midten' 'larsloekke' 'verdenskrig' 'partilederdebat'
 'forventninger' 'sikkerhedspolitik' 'fremtiden' 'nytænkning'
 'folkepension' 'organisering' 'omstille' 'lmå' 'skattepolitik' 'diabetes'
 'fremlagde' 'forandring' 'reformer' 'valgdag' 'fe-sagen' 'blokpolitik'
 'mortenhelveg' 'brydfri' 'løse' 'kandidater' 'selvstændig' 'egentlige'
 'partiets' 'gæst' 'hen' 'skræmmekampagne' 'tænknyt' 'jonasholmdk'
 'efterløn' 'rasmussen' 'ambitionen' 'fjernvarme' 'hovedstad'
 'udfordringerne' '🧵' 'politikens' 'inspirere' 'løkke' 'ii' 'henover'
 'centrum'] 

--- TOP 1 DOCS. SUBSTRING QUERY: None (n = 1) ---
 ['Læs hele vores reformforslag her:\xa0\nhttps://t.co/Y7qlZdVB5S #dkpol https://t.co/oATqigD5u5. RT @LarsSandahlS: Interessante forslag fra @moderaterne_dk og @larsloekke. Bl.a ønsket om lavere kapitalskat vil styrke #dkbiz\'s muligheder…. RT @BrianAMikkelsen: Der er brug for alle gode idéer, hvis v

In [85]:
model_path = 'embedding_models/politician2vec_test_v1.txt'

In [86]:
pol2vec_model.save(model_path)

### Viz dev below

In [87]:
pol2vec_model, doc2vec_model = load_politician2vec_from_txt(model_path)

Loading Politician2Vec model...
Retrieving document embedding...
All done!


In [103]:
doc2vec2tensor(
    doc2vec_model,
    temp_w2v_path = 'tensorboard_input/temp/doc_tensor_no_wv.w2v',
    tsv_prefix = 'tensorboard_input/TENSOR_TEST_V1',
    output_docvecs = True,
    output_wordvecs = False
)

You have elected to extract only document vectors.
Please note that further preprocessing -- such as filtering based on topics of
interest -- may be desired in order to facilitate TensorBoard visualisation.
Please see get_doc_topic_df(), vector_subset2tensor_without_words(), and
metadata2tensor()

Saving temp w2v file and converting to tensor. This may take a while...


2022-11-12 10:03:52,778 - word2vec2tensor - INFO - running /Users/mathiasbruun/me/anaconda3/lib/python3.7/site-packages/gensim/scripts/word2vec2tensor.py -i tensorboard_input/temp/doc_tensor_no_wv.w2v -o tensorboard_input/TENSOR_TEST_V1
2022-11-12 10:03:52,779 - keyedvectors - INFO - loading projection weights from tensorboard_input/temp/doc_tensor_no_wv.w2v
2022-11-12 10:03:52,820 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (154, 300) matrix of type float32 from tensorboard_input/temp/doc_tensor_no_wv.w2v', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-11-12T10:03:52.802900', 'gensim': '4.1.2', 'python': '3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 05:57:50) \n[Clang 11.1.0 ]', 'platform': 'Darwin-21.4.0-x86_64-i386-64bit', 'event': 'load_word2vec_format'}
2022-11-12 10:03:52,872 - word2vec2tensor - INFO - 2D tensor file saved to tensorboard_input/TENSOR_TEST_V1_tensor.tsv
2022-11-12 10:03:52,872 - word2vec2tensor - INFO - Tensor metadata file 

In [88]:
n_words = len(doc2vec_model.wv)
n_docs = len(doc2vec_model.dv)
vocab = pol2vec_model.vocab

In [89]:
topic_labels = {
    0: 'Parti 1',
    1: 'Parti 2',
    2: 'Parti 3',
    3: 'Parti 4',
    4: 'Parti 5',
    5: 'Parti 6',
    6: 'Parti 7',
    7: 'Parti 8',
    8: 'Parti 9',
    9: 'Parti 10',
    10: 'Parti 11',
    11: 'Parti 12',
    12: 'Parti 13'
}

In [121]:
topic_df = get_doc_topic_df(
    pol2vec_model,
    no_substantive_topics = 13,
    snippets = True
)

Topic sizes before filtering (topic 13 is "Other"):

[[ 0 35]
 [ 1 31]
 [ 2 16]
 [ 3 14]
 [ 4 13]
 [ 5 11]
 [ 6 10]
 [ 7  6]
 [ 8  5]
 [ 9  5]
 [10  5]
 [11  2]
 [12  1]]


In [122]:
topic_df

,doc,top,snippet
0,0,7,Det offentlige forbrug er steget med 9 mia. kr...
1,1,8,RT @TFKortholm: Vi er klar til valgkampen 💚🖖 S...
2,2,8,Vores super seje @OlumekoC er på den internati...
3,3,1,RT @DSU_1920: Statsministeren underskriver DSU...
4,4,4,"Hvad er egentlig @SorenPape, @KonservativeDK, ..."
...,...,...,...
149,149,0,RT @JakobEllemann: Danskerne drømmer om at bo ...
150,150,0,RT @AndreasBachM: 75.135 ‼️ gange tak til alle...
151,151,0,En borgerlig statsminister forudsætter et borg...
152,152,1,"@JackPGB Ja, man får det dårligt 😔. Det her gø..."


In [123]:
# TODO: Okay, we have clearly made a mistake. By swapping party classes for politicians,
# we are now telling the model to find politician-level centroids wrt. politician superdocs, yielding...
# ... well, obviously, 154 centroids, correspodning to the 154 super documents. That makes no sense,
# since the centroid and the superdocument will be the same by construction.
#
# We either want document-level centroids clustered by politician OR politician-level centroids clustered
# by party. So politician superdocs + input party label array.
#
# TODO: Also extract topic vectors themselves, which we usually never do!
#
# TODO: Perhaps chunk input superdocs according to certain special character that we join them on initially

In [124]:
topic_df

,doc,top,snippet
0,0,7,Det offentlige forbrug er steget med 9 mia. kr...
1,1,8,RT @TFKortholm: Vi er klar til valgkampen 💚🖖 S...
2,2,8,Vores super seje @OlumekoC er på den internati...
3,3,1,RT @DSU_1920: Statsministeren underskriver DSU...
4,4,4,"Hvad er egentlig @SorenPape, @KonservativeDK, ..."
...,...,...,...
149,149,0,RT @JakobEllemann: Danskerne drømmer om at bo ...
150,150,0,RT @AndreasBachM: 75.135 ‼️ gange tak til alle...
151,151,0,En borgerlig statsminister forudsætter et borg...
152,152,1,"@JackPGB Ja, man får det dårligt 😔. Det her gø..."


In [125]:
topic_df['snippet'] = grouped_docs['full_name']

In [126]:
topic_df

,doc,top,snippet
0,0,7,Alex Vanopslagh
1,1,8,Alternativet
2,2,8,Alternativets Unge
3,3,1,Anders Kronborg
4,4,4,Andreas Steenberg
...,...,...,...
149,149,0,Ulla Tørnæs
150,150,0,Venstre
151,151,0,Venstres Ungdom
152,152,1,Yildiz Akdogan


In [127]:
len(pol2vec_model.topic_vectors)

13

In [128]:
metadata2tensor(
    topic_df,
    metadata_path = 'tensorboard_input/TENSOR_TEST_V1_METADATA.tsv',
    label_list = topic_labels
)

In [ ]:
#vector_subset2tensor_without_words(
#    topic_df,
#    orig_vec_path = 'tensorboard_input/TENSOR_TEST_V1',
#    out_path = 'tensorboard_input/TENSOR_TEST_V1'
#)